In [60]:
import logging
logging.getLogger().setLevel(logging.WARNING)

from konlpy.corpus import kobill, kolaw
from konlpy.tag import Twitter, Kkma
twitter = Twitter()
kkma = Kkma()

import string
import re
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
remove_quotes_map = dict([(ord(x), " ") for x in ".․,‘’´“”·‧<>「」–-()~…"]) 
def clean_title(title):
    # Remove punctuation and quotes
    title = title.translate(remove_punct_map)
    title = title.translate(remove_quotes_map)
    # Collapse double spaces
    title = re.sub('\s+', ' ', title)
    return title

stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
#stopwords += "공급 설치 조성 운영 실행 설립 확대 제공 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
#stopwords += "을 것 ㄴ ㄹ 과 로 는 하".split()
stopwords = set(stopwords)

pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Alpha', 'Adjective')]
pos = lambda d: [p for p in kkma.morphs(clean_title(d)) if p not in stopwords]

In [61]:
twitter.pos('청각장애인과 함께 CCTV로 만드는 안전한 서울', stem=True, norm=True)

[('청각장애', 'Noun'),
 ('인과', 'Noun'),
 ('함께', 'Adverb'),
 ('CCTV', 'Alpha'),
 ('로', 'Noun'),
 ('만들다', 'Verb'),
 ('안전하다', 'Adjective'),
 ('서울', 'Noun')]

In [63]:
pos('시민들께 서울의 혁신점수를 알려드려요')

['시민', '들', '께', '서울', '의', '혁신', '점수', '를', '알리', '어', '드리', '어요']

In [54]:
%%time
texts_kobill = [pos(kobill.open(i).read()) for i in kobill.fileids()]
texts_kolaw = [pos(kolaw.open(i).read()) for i in kolaw.fileids()]

CPU times: user 8.52 s, sys: 316 ms, total: 8.84 s
Wall time: 4.77 s


In [55]:
%%time
import json

texts = []

promise_texts = []
for idx in range(250):
    promise_texts.append(open('../promisedata/korean/prom%d.txt' % (idx+1)).read())

idx = 0
for fname in ('jisa_tagged.json', ): # 'mayor_tagged.json',
    with open(fname) as data_file:    
        data = json.load(data_file)
        for voting_district in data:
            if voting_district['city'] != 'seoul' or voting_district['district'] > 28:
                continue
            promises = voting_district['promises']
            for p in promises:
                text = p['title']
                if idx < 250:
                    text += "\n" + promise_texts[idx]
                texts.append((p['title'], pos(text), (voting_district['city'], voting_district['district'])))
                idx += 1


texts_promises = [text[1] for text in texts]
print(len(texts_promises))

with open('test-data/promise-titles.json', 'w') as f:
    json.dump([text[0] for text in texts], f)
    


250
CPU times: user 20.4 s, sys: 386 ms, total: 20.8 s
Wall time: 26 s


In [56]:
from gensim import corpora, models
# Create dictionary
corpus = texts_kobill + texts_kolaw + texts_promises
#corpus = texts_promises
logging.info('Dictionary from %d documents' % len(corpus))
dictionary_ko = corpora.Dictionary(corpus)
# Filter terms that occur in more than 10% of the documents
dictionary_ko.filter_extremes(no_below=0, no_above=0.1)
# Save for later use
dictionary_ko.save('ko.dict')

In [58]:
%%time
from gensim import similarities

num_topics = 22

# Generate Lsi model for promise text corpus
corpus = [dictionary_ko.doc2bow(text) for text in texts_promises]
# Train Lsi model
lsi = models.LsiModel(corpus, id2word=dictionary_ko, num_topics=num_topics)
lsi.save('lsi.model')
# Transform corpus to LSI space and index it
lsi_corpus = lsi[corpus]
index = similarities.MatrixSimilarity(lsi_corpus)
index.save('lsi.index')

CPU times: user 1.02 s, sys: 176 ms, total: 1.2 s
Wall time: 1.29 s


In [37]:
def find_similar(tagged_text, similarity_threshold=None):
    # Convert text to bag of words
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    # Convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    # Perform a similarity query against the corpus
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    # Return the similarities sorted by descending score
    return sorted(sims, key=lambda item: -item[1])
    
# Try to match a few existing documents

for text in texts[100:101]:
    sims = find_similar(text[1], similarity_threshold=0.75)
    print("Most similar to %s" % (text[0],))
    for key, score in sims[1:]:
        key = int(key)
        print('%.2f - %s' % (score, texts[key][0]))


Most similar to 동북아 대도시 대기환경협의체 구성 및 대기개선 노하우 공유
0.84 - 질 높은 인터넷 강의 개설
0.84 - 2) 생태환경중심의 서대문 도시발전계획 수립
0.83 - 질 높은 영어교육 실시
0.83 - 녹색도시 생활환경기반 구축
0.82 - 다중이용시설 실내공기질 개선
0.81 - 강동 친환경 도시농업 2020 프로젝트
0.81 - 동자동, 용산역, 국제빌딩주변 도시환경정비사업
0.81 - 서울-세종 간 고속도로 강동통과 반대 지속 협의
0.79 - 신대방역~보라매타운 간 난곡GRT 지속추진
0.77 - 2) 충정로권역 업무중심 지역개발 (도시환경정비사업 추진)


In [39]:
%%time
import numpy as np
# Find similarities of all documents with each other and see how many matches higher than x score there are
# Used to figure out a threshold that regards enough documents as matched
threshold = 0.8
counts = []
counts_self_no_match = []
for idx, text in enumerate(texts[:500]):
    similarities = np.array(find_similar(text[1]))
    # Remove self-similarity
    id_idx = np.where(similarities[:, 0]==idx)[0][0]
    similarities = np.delete(similarities, id_idx, axis=0)
    # Record when self wasn't the best match (it is usally the 2nd best in these cases)
    if id_idx != 0:
        counts_self_no_match.append((idx, id_idx))
    # Count number of docs with similarity >= threshold
    count = np.sum(similarities[:,1]>=threshold)
    counts.append(count)

counts = np.array(counts)
counts_self_no_match = np.array(counts_self_no_match)
print('Number of matches per document: max %d, min %d, mean %.2f, median %d' % (np.max(counts), np.min(counts), np.mean(counts), np.median(counts)))
print('%d documents didn\'t match with themself as their best match' % (len(counts_self_no_match)))
if len(counts_self_no_match):
    print('  but %d of them matched at pos 1.' % (np.sum(counts_self_no_match[:,1]==1)))

no_match_count = np.sum(counts==0)
print('%d (%d%%) with 0 matches' % (no_match_count, 100*no_match_count/len(counts)))

Number of matches per document: max 63, min 0, mean 18.32, median 16
5 documents didn't match with themself as their best match
  but 4 of them matched at pos 1.
25 (5%) with 0 matches
CPU times: user 2.06 s, sys: 63.2 ms, total: 2.12 s
Wall time: 3.23 s


In [50]:
print("Document with highest number of matches:")
print(texts[np.argmax(counts)])

Document with highest number of matches:
('테마교육공원 4개소 조성', ['테마/Noun', '교육/Noun', '공원/Noun', '개다/Verb', '테마/Noun', '교육/Noun', '공원/Noun', '개다/Verb', '상황/Noun', '이행/Noun', '후/Noun', '계속/Noun', '사업/Noun', '구분/Noun', '임기/Noun', '내/Noun', '계/Noun', '속/Noun', '총/Noun', '사업/Noun', '비/Noun', '백만원/Noun', '집/Noun', '행/Noun', '액/Noun', '백만원/Noun', '최종/Noun', '목표/Noun', '기존/Noun', '공원/Noun', '인프라/Noun', '활용/Noun', '하다/Verb', '미래/Noun', '지향/Noun', '학습/Noun', '체험/Noun', '활동/Noun', '지역/Noun', '특성/Noun', '반영/Noun', '하다/Verb', '대다/Verb', '권역/Noun', '별로/Noun', '동부/Noun', '어린이/Noun', '청소년/Noun', '문화/Noun', '체험/Noun', '교육/Noun', '공원/Noun', '어린이/Noun', '공원/Noun', '서부/Noun', '에너지/Noun', '환경/Noun', '교육/Noun', '공원/Noun', '월드컵/Noun', '공원/Noun', '남부/Noun', '다하다/Verb', '자연/Noun', '하나/Noun', '되다/Verb', '생태/Noun', '문화/Noun', '교육/Noun', '공원/Noun', '서울대/Noun', '공원/Noun', '북부/Noun', '문화/Noun', '예술/Noun', '역사/Noun', '흐르다/Verb', '교육/Noun', '공원/Noun', '북서울꿈의숲/Noun', '공원/Noun', '중부/Noun', '나비/Noun', '곤충/Noun', '테마/Noun',

In [47]:
print("Unmatched documents:")
for idx in np.where(counts==0)[0]:
    print(texts[idx][0], texts[idx][1])

Unmatched documents:
개포모바일 융합클러스터 조성사업 추진 ['개포/Noun', '모바일/Noun', '융합/Noun', '클러스터/Noun', '사업/Noun']
시장 직속 일자리 위원회 신설 ['시장/Noun', '직속/Noun', '일자리/Noun', '위원회/Noun', '신설/Noun']
한양도성 세계유산 등재 ['한양도성/Noun', '세계/Noun', '유산/Noun', '등재/Noun']
잠자리, 먹거리, 일거리가 한곳에서 해결되도록 지원 ['잠자리/Noun', '먹거리/Noun', '일거리/Noun', '곳/Noun', '해결/Noun', '되다/Verb']
불공정피해 상담센터 기능 확대 ['불공정/Noun', '피해/Noun', '상담/Noun', '센터/Noun', '기능/Noun']
서울시네마테크’ 건립 ['서울/Noun', '시네마테크/Noun', '건립/Noun']
지하철역 엘리베이터 설치 지속추진, 공공시설 장애인 동선 확보 ['지하철역/Noun', '엘리베이터/Noun', '지속/Noun', '공공시설/Noun', '장애인/Noun', '동선/Noun', '확보/Noun']
수도권연계 광역도시철도 신설 및 제3기 도시철도 조기 추진 ['수도/Noun', '권연/Noun', '광역/Noun', '도시철도/Noun', '신설/Noun', '제/Noun', '기/Noun', '도시철도/Noun', '조기/Noun']
서울 선데이파크에서 놀면서 살빼요 ['서울/Noun', '선데이/Noun', '파크/Noun', '놀다/Verb', '살빼다/Adjective']
사고위치 번호판 운영 ['사고/Noun', '위치/Noun', '번호판/Noun']
글로벌 의료관광 육성 ['글로벌/Noun', '의료관광/Noun']
금융기관 본점 등 건실한 기업 유치 ['금융기관/Noun', '본점/Noun', '건실/Noun', '기업/Noun']
자원봉사와 노인일자리 등 생산적인 노후생활 활성화 ['자원봉사/Noun', '노인/Noun', 

In [27]:
# Try some queries that don't match existing documents

def match(title):
    tags = pos(title)
    print("Most similar to %s (%s)" % (title, " ".join(tags)))
    threshold = 0.75
    for posi, item in enumerate(find_similar(tags, threshold)):
        key = int(item[0])
        score = item[1]
        print('%2d. %.2f - %s - %s %d' % (posi+1, score, texts[key][0], texts[key][2][0], texts[key][2][1]))

match('임대주택')
match(' 임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토')
match('지난해 저소득층 근로소득 감소에…소득 불평등도 악화 : 네이버 뉴스')
match('노후자금 털어 집에 투자? 전세금, 집값 용도 퇴직연금 중도인출 급증 : 네이버 뉴스')

text = "대학 등록금 동결 속 외국인 유학생만 '봉' 서울 주요 대학들이 외국인 유학생이 내야 할 등록금을 잇달아 올리려 하는 것으로 드러났다. 내국인 대학생에 대한 ‘등록금 동결’과 ‘입학금 폐지’로 줄어든 수입을 외국인 유학생에게서 더 받아내 만회하겠다는 것이다."
match(text)

Most similar to 임대주택 (임대 주택)
Most similar to  임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토 (임대 주택 등록 하면 세금 · 건 보료 감면 … 2020 등록 의무화)
 1. 0.99 - 민간임대주택 지원을 위한 국민주택기금 및 이자지원 확대 - seoul 0
 2. 0.99 - 국공립어린이집 1,000개소 확충 - seoul 0
 3. 0.98 - 시민 모니터링요원 및 전담조직 신설 - seoul 0
 4. 0.97 - 안심주택 8만호 공급(2018년) - seoul 0
 5. 0.96 - 함께나눠쓰는공구,서울시공구은행 - seoul 0
 6. 0.94 - 불공정피해 상담센터 기능 확대 - seoul 0
 7. 0.93 - 대학가 주변 도전숙 조성 - seoul 0
 8. 0.93 - 관리형 및 공공토지임대형 주택협동조합 육성 - seoul 0
 9. 0.92 - 세대융합(화합)형 공공주택 공급 - seoul 0
10. 0.87 - 개포모바일 융합클러스터 조성사업 추진 - seoul 0
11. 0.85 - 저렴 소형주택 집중공급을 위한 도시계획 지원강화 - seoul 0
12. 0.85 - 전용면적 30~60m² 소형주택 20만호 공급 지원 - seoul 0
Most similar to 지난해 저소득층 근로소득 감소에…소득 불평등도 악화 : 네이버 뉴스 (지난해 저소득 층 근로 소득 감소 에 … 소득 불평등 도 악화 :)
Most similar to 노후자금 털어 집에 투자? 전세금, 집값 용도 퇴직연금 중도인출 급증 : 네이버 뉴스 (노후 자금 털 어 집 에 투자 ? 전세금 , 집값 용도 퇴직 연금 중도 인출 급증 :)
 1. 0.88 - 자영업 지원센터’ 설치 - seoul 0
 2. 0.88 - 우리 동네 거리에 과일이 넘쳐요 - seoul 0
 3. 0.86 - 저학년 초등학생 통학거리 안전보장 - seoul 0
 4. 0.86 - 우리 아이 돌보미, 서울시가 찾아드려요 - seoul 0
 5. 0.

In [28]:
corpora.MmCorpus.serialize('lsi.mm', lsi_corpus)

In [29]:
%%time
import numpy as np
from sklearn.manifold import TSNE
from gensim import matutils

lsi_corpus = corpora.MmCorpus('lsi.mm')
tsne = TSNE(n_components=2, init='pca', random_state=0, metric='cosine', angle=.99)
X = matutils.corpus2dense(lsi_corpus, num_terms=lsi_corpus.num_terms)
X_data = np.asarray(X).astype('float64')
X_tsne = tsne.fit_transform(X_data.T)

CPU times: user 4.48 s, sys: 692 ms, total: 5.17 s
Wall time: 6.8 s


In [30]:
# Only use topics with high enough propability
threshold = 0.1
_idx = np.amax(X, axis=0) > threshold
X_ = X[:, _idx]
X_tsne_ = X_tsne[_idx, :]

In [31]:
#X_tsne[_idx, 0].shape
X_.shape

(20, 156)

In [32]:
import matplotlib
matplotlib.rc('font', family='NanumGothic')

from matplotlib import pyplot as plt
top_topics = np.argmax(X_, axis=0)

topics_words = lsi.show_topics(num_words=5, formatted=False)
topic_summaries = [' '.join([word[0].split('/')[0] for word in topic]) for topic_no, topic in topics_words]

plt.figure(figsize=(15,10))
plt.scatter(X_tsne_[:, 0], X_tsne_[:, 1], c=top_topics, cmap=plt.cm.get_cmap("jet", 50))

plt.colorbar(ticks=range(50))
plt.clim(-0.5, 9.5)

# find center of topic clusters
topic_coord = np.empty((len(topic_summaries), 2)) * np.nan
for topic_num in range(len(topic_summaries)):
    if not np.isnan(topic_coord).any():
        break
    docs_in_topics = (top_topics==topic_num)
    if not docs_in_topics.any():
        # no documents were assigned to this topic
        continue
    #topic_coord[topic_num] = X_tsne_[top_topics==topic_num].mean(axis=0)
    topic_coord[topic_num] = X_tsne_[np.argmax(top_topics==topic_num)]

# plot topic summaries
for i in range(len(topic_summaries)):
    if np.isnan(topic_coord[i]).any():
        continue
    plt.text(topic_coord[i, 0], topic_coord[i, 1], topic_summaries[i])


#plt.show()
plt.savefig('tsne.png')

In [33]:
X_.shape

(20, 156)

In [34]:
import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import output_file, save
from bokeh.models import HoverTool
from random import shuffle

colors = ["#E495A5", "#E3979D", "#E19895", "#DE9A8E", "#DB9C86", "#D89F7F", "#D3A178", "#CEA372", "#C9A66D", "#C3A869", "#BDAB66", "#B6AD65", "#AFAF64", "#A7B166", "#9EB368", "#96B56C", "#8DB771", "#83B977", "#79BA7E", "#6FBB84", "#65BC8C", "#5ABD93", "#50BE9B", "#46BEA2", "#3FBEAA", "#39BEB1", "#37BDB8", "#3ABCBF", "#41BBC5", "#4ABACB", "#55B8D0", "#61B6D5", "#6DB4D9", "#79B1DC", "#85AEDF", "#91ACE1", "#9CA9E2", "#A6A6E2", "#B0A3E1", "#B9A0E0", "#C29DDE", "#C99ADB", "#D098D7", "#D596D3", "#DA95CD", "#DE94C8", "#E093C1", "#E393BB", "#E493B4", "#E494AC"]
shuffle(colors)
colormap = np.array(colors)
title = 'Promises LSI vizualization'
num_example = len(X_)

plot = bp.figure(plot_width=1200, plot_height=700,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     min_border=1)

plot.scatter(x='x', y='y',
                 color='color',
                 source=bp.ColumnDataSource({
                   "x": X_tsne_[:, 0], "y": X_tsne_[:, 1],
                   "content": np.array(['%s %s %d' % (text[0], text[2][0], text[2][1]) for text in texts])[_idx],
                   "topic_key": top_topics,
                   "topic_summary": [topic_summaries[i] for i in top_topics],
                   "color": colormap[top_topics]
                   }))

for i in range(len(topic_summaries)):
    if np.isnan(topic_coord[i]).any():
        continue
    plot.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

    
hover = plot.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key @topic_summary"}
output_file('{}.html'.format(title))
save(plot)


'/Users/paul/Projects/kixlab/xproject/xproj-utils/promise-match/Promises LSI vizualization.html'